In [5]:
import torch
import datetime
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.models import squeezenet, alexnet, inception_v3
from torchvision.datasets import coco
from torchvision import transforms

In [6]:
def compute_mem_size(t):
    t_size = t.size()
    n_nums = 1
    for n in t_size:
        n_nums *= n
    bit_size_map = {torch.float32 : 32, torch.float16 : 16, torch.float64 : 64, torch.uint8 : 8, torch.int8 : 8, torch.int16 : 16, torch.int32 : 32, torch.int64 : 64}
    return (n_nums * bit_size_map[t.dtype]) / 8

In [7]:
BASE = '/Users/peterspradling/CS349D'
ANN_PATH = '/coco/cocoapi/annotations/instances_val2017.json'
VAL_PATH = '/coco/cocoapi/images/val2017/'

In [8]:
def merge_coco(l):
    return torch.stack([x[0] for x in l])

## Squeezenet

In [10]:
squeezenet_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225])
])
squeezenet_dataset = coco.CocoDetection(root=BASE+VAL_PATH, annFile=BASE+ANN_PATH, transform=squeezenet_transform)
squeezenet_eval_loader = torch.utils.data.DataLoader(squeezenet_dataset, batch_size=1, shuffle=True, collate_fn=merge_coco)

loading annotations into memory...
Done (t=0.86s)
creating index...
index created!


In [11]:
squeezenet_model = squeezenet.squeezenet1_0(pretrained=True)

/Users/peterspradling/anaconda3/envs/torch/lib/python3.5/site-packages/torchvision-0.2.1-py3.5.egg/torchvision/models/squeezenet.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
/Users/peterspradling/anaconda3/envs/torch/lib/python3.5/site-packages/torchvision-0.2.1-py3.5.egg/torchvision/models/squeezenet.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
Downloading: "https://download.pytorch.org/models/squeezenet1_0-a815701f.pth" to /Users/peterspradling/.torch/models/squeezenet1_0-a815701f.pth
100.0%


In [19]:
def squeezenet_measure_forward(model, x):
    print('Image size:', compute_mem_size(x))
    for module in model.features:
        t1 = datetime.datetime.now()
        x = module(x)
        t2 = datetime.datetime.now()
        dt1 = t2 - t1
        canon_name = type(module).__name__
        print('After ' + canon_name + '(' + str(dt1.total_seconds()) + '):  ' + str(compute_mem_size(x)))
    # resize step.
    #x = x.view(x.size(0), 256 * 6 * 6)
    # classification step
    for module in model.classifier:
        t1 = datetime.datetime.now()
        x = module(x)
        t2 = datetime.datetime.now()
        dt1 = t2 - t1
        canon_name = type(module).__name__
        print('After ' + canon_name + '(' + str(dt1.total_seconds()) + '):  ' + str(compute_mem_size(x)))
    return x

In [20]:
squeezenet_model.eval()
for batch in squeezenet_eval_loader:
    squeezenet_measure_forward(squeezenet_model, batch)
    break

Image size: 602112.0
After Conv2d(0.007228):  4562304.0
After ReLU(0.002917):  4562304.0
After MaxPool2d(0.012698):  1119744.0
After Fire(0.043687):  1492992.0
After Fire(0.018239):  1492992.0
After Fire(0.014437):  2985984.0
After MaxPool2d(0.013278):  746496.0
After Fire(0.008686):  746496.0
After Fire(0.005105):  1119744.0
After Fire(0.005788):  1119744.0
After Fire(0.005746):  1492992.0
After MaxPool2d(0.003027):  346112.0
After Fire(0.002068):  346112.0
After Dropout(9.3e-05):  346112.0
After Conv2d(0.002455):  676000.0
After ReLU(0.000431):  676000.0
After AvgPool2d(0.000298):  4000.0


## AlexNet

In [ ]:
alexnet_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(227),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225])
])
alexnet_dataset = coco.CocoDetection(root=BASE+VAL_PATH, annFile=BASE+ANN_PATH, transform=resnet_transform)
alexnet_eval_loader = torch.utils.data.DataLoader(alexnet_dataset, batch_size=1, shuffle=True, collate_fn=merge_coco)

In [ ]:
alexnet_model = alexnet(pretrained=True)

In [ ]:
def alexnet_measure_forward(model, x):
    print('Image size: ', compute_mem_size(x))
    for module in model.features:
        t1 = datetime.datetime.now()
        x = module(x)
        t2 = datetime.datetime.now()
        dt1 = t2 - t1
        canon_name = type(module).__name__
        print('After ' + canon_name + '(' + str(dt1.total_seconds()) + '):  ' + str(compute_mem_size(x)))
    # resize step.
    x = x.view(x.size(0), 256 * 6 * 6)
    # classification step
    for module in model.classifier:
        t1 = datetime.datetime.now()
        x = module(x)
        t2 = datetime.datetime.now()
        dt1 = t2 - t1
        canon_name = type(module).__name__
        print('After ' + canon_name + '(' + str(dt1.total_seconds()) + '):  ' + str(compute_mem_size(x)))

In [ ]:
alexnet_model.eval()
for batch in alexnet_eval_loader:
    alexnet_measure_forward(alexnet_model, batch)
    break

## Inception V3

In [6]:
inception_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225])
])
inception_dataset = coco.CocoDetection(root=BASE+VAL_PATH, annFile=BASE+ANN_PATH, transform=inception_transform)
inception_eval_loader = torch.utils.data.DataLoader(inception_dataset, batch_size=1, shuffle=True, collate_fn=merge_coco)

loading annotations into memory...
Done (t=0.65s)
creating index...
index created!


In [7]:
inception_model = inception_v3(pretrained=True)

In [10]:
# incomplete
def inception_measure_forward(model, x):
    print('Image size: ', compute_mem_size(x))
    # 299 x 299 x 3
    t1 = datetime.datetime.now()
    x = model.Conv2d_1a_3x3(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After Conv2d1a (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 149 x 149 x 32
    t1 = datetime.datetime.now()
    x = model.Conv2d_2a_3x3(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After Conv2d2a (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 147 x 147 x 32
    t1 = datetime.datetime.now()
    x = model.Conv2d_2b_3x3(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After Conv2d2b (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 147 x 147 x 64
    t1 = datetime.datetime.now()
    x = F.max_pool2d(x, kernel_size=3, stride=2)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After MaxPool2d (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 73 x 73 x 64
    t1 = datetime.datetime.now()
    x = model.Conv2d_3b_1x1(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After Conv2d3B (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 73 x 73 x 80
    t1 = datetime.datetime.now()
    x = model.Conv2d_4a_3x3(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After Conv2d4A (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 71 x 71 x 192
    t1 = datetime.datetime.now()
    x = F.max_pool2d(x, kernel_size=3, stride=2)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After MaxPool2D (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 35 x 35 x 192
    t1 = datetime.datetime.now()
    x = model.Mixed_5b(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After Mixed5B (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 35 x 35 x 256
    t1 = datetime.datetime.now()
    x = model.Mixed_5c(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After Mixed5C (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 35 x 35 x 288
    t1 = datetime.datetime.now()
    x = model.Mixed_5d(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After Mixed5D (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 35 x 35 x 288
    t1 = datetime.datetime.now()
    x = model.Mixed_6a(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After Mixed6A (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 17 x 17 x 768
    t1 = datetime.datetime.now()
    x = model.Mixed_6b(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After Mixed6B (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 17 x 17 x 768
    t1 = datetime.datetime.now()
    x = model.Mixed_6c(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After Mixed6C (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 17 x 17 x 768
    t1 = datetime.datetime.now()
    x = model.Mixed_6d(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After Mixed6D (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 17 x 17 x 768
    t1 = datetime.datetime.now()
    x = model.Mixed_6e(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After Mixed6E (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 17 x 17 x 768
    t1 = datetime.datetime.now()
    x = model.Mixed_7a(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After Mixed7A (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 8 x 8 x 1280
    t1 = datetime.datetime.now()
    x = model.Mixed_7b(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After Mixed7B (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 8 x 8 x 2048
    t1 = datetime.datetime.now()
    x = model.Mixed_7c(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After Mixed7C (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 8 x 8 x 2048
    t1 = datetime.datetime.now()
    x = F.avg_pool2d(x, kernel_size=8)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After AvgPool2d (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 1 x 1 x 2048
    x = F.dropout(x, training=model.training)
    # 1 x 1 x 2048
    x = x.view(x.size(0), -1)
    # 2048
    t1 = datetime.datetime.now()
    x = model.fc(x)
    t2 = datetime.datetime.now()
    dt1 = t2 - t1
    print('After FC (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    # 1000 (num_classes)
    return x

In [11]:
inception_model.eval()
for batch in inception_eval_loader:
    inception_measure_forward(inception_model, batch)
    break

Image size:  1072812.0
After Conv2d1a (0.034189): 2841728.0
After Conv2d2a (0.024898): 2765952.0
After Conv2d2b (0.032337): 5531904.0
After MaxPool2d (0.018495): 1364224.0
After Conv2d3B (0.010574): 1705280.0
After Conv2d4A (0.024739): 3871488.0
After MaxPool2D (0.009854): 940800.0
After Mixed5B (0.026174): 1254400.0
After Mixed5C (0.035077): 1411200.0
After Mixed5D (0.047889): 1411200.0
After Mixed6A (0.026884): 887808.0
After Mixed6B (0.035302): 887808.0
After Mixed6C (0.038276): 887808.0
After Mixed6D (0.039578): 887808.0
After Mixed6E (0.044512): 887808.0
After Mixed7A (0.017383): 327680.0
After Mixed7B (0.023297): 524288.0
After Mixed7C (0.029244): 524288.0
After AvgPool2d (0.000436): 8192.0
After FC (0.007548): 4000.0
